In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime
import pandas as pd

In [ ]:
!which chromedriver

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [/Users/abby/.wdm/drivers/chromedriver/mac64/83.0.4103.39/chromedriver] found in cache


In [3]:
base_url = 'https://www.walmart.com'
search_url = '/search/?query=room%20air%20purifier'
browser.visit(base_url + search_url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
products = soup.find_all('div', class_='search-result-gridview-item')

In [6]:
# Create empty lists for scraped data to be stored in.
productURL_list = []
productImage_list = []
productTitle_list = []
starReview_list = []
reviewAmount_list = []
currentPrice_list = []
reviewURL_list = []


today = datetime.datetime.now()

for product in products:

    # Get the link to the product page.
    link = product.find('a')
    href = link['href']
    product_url = base_url + href
    productURL_list.append(product_url)

    # Get the link to the product image.
    img = product.find('img')['src']
    productImage_list.append(img)

    # Get the product title.
    product_title = product.find('img')['alt']
    productTitle_list.append(product_title)

    # Try to get the number of reviews, there may not be any.
    try:
        review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
        reviewAmount_list.append(review_amount)
        
    except:
        review_amount = 0
        reviewAmount_list.append(review_amount)

    # In the case where there are at least one review...
    if int(review_amount) > 0:
        # Get the average number of stars
        stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
        starReview_list.append(stars_review)
        
        # Get the URL to the reviews section for that product.
        review_url = product.find('div', class_='stars').find('a')['href']
        reviewURL_list.append(review_url)

    # Otherwise, use defalt 0 or NaN values for these entries.
    else:
        stars_review = 0
        starReview_list.append(stars_review)
        
        review_url = "NaN"
        reviewURL_list.append(review_url)
        
    # Get the price of the product.
    price = product.find('span', class_='price-main')
    current_price = price.find('span', class_='visuallyhidden').text
    currentPrice_list.append(current_price)
    
# Close the browser window
# browser.quit()
    
# Create a dictionary with the lists of the scrapped data.
data = {
    "Title": productTitle_list,
    "URL": productURL_list,
    "Image": productImage_list,
    "AverageStars": starReview_list,
    "NumberofReviews": reviewAmount_list,
    "ReviewsURL": reviewURL_list,
    "Price": currentPrice_list
       }

# Create a Pandas DataFrame with that dictionary
product_df = pd.DataFrame.from_dict(data)
product_df.head()

,Title,URL,Image,AverageStars,NumberofReviews,ReviewsURL,Price
0,"Filtrete by 3M Room Air Purifier, Console, 110...",https://www.walmart.com/ip/Filtrete-by-3M-Room...,https://i5.walmartimages.com/asr/eae28c2f-5dbe...,4.6,185,/ip/Filtrete-by-3M-Room-Air-Purifier-Console-1...,$49.94
1,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,https://i5.walmartimages.com/asr/ca7e3b65-d3c8...,4.3,513,/ip/Holmes-Personal-Space-2-Speed-Air-Purifier...,$14.98
2,Winix 5500-2 Air Cleaner with PlasmaWave Techn...,https://www.walmart.com/ip/Winix-5500-2-Air-Cl...,https://i5.walmartimages.com/asr/7e1f8414-d0c2...,4.7,86,/ip/Winix-5500-2-Air-Cleaner-with-PlasmaWave-T...,$159.99
3,High-Efficiency HEPA-Type Desktop Air Purifier...,https://www.walmart.com/ip/High-Efficiency-HEP...,https://i5.walmartimages.com/asr/5ba9740e-a3f1...,4.4,136,/ip/High-Efficiency-HEPA-Type-Desktop-Air-Puri...,$49.69
4,TENDOMI WiFi Smart Air Purifier with True HEPA...,https://www.walmart.com/ip/TENDOMI-WiFi-Smart-...,https://i5.walmartimages.com/asr/a4b27232-85c0...,4.5,20,/ip/TENDOMI-WiFi-Smart-Air-Purifier-True-HEPA-...,$65.99


In [9]:
# Get reviews 

review_df = pd.DataFrame()

for i in range(len(product_df["ReviewsURL"])):
    try:
        browser.visit(base_url+product_df["ReviewsURL"][i])
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
        browser.visit(base_url+allReviewsURL)
        soup = BeautifulSoup(html, 'html.parser')
        reviews = soup.find_all('div', class_='ReviewList-content')

        # need to add a way to make sure I am capturing all of the reviews
        # can check this by looking at NumberofReview for this ReviewsRUL
        # to scroll down a little use the below command
        # browser.execute_script("window.scrollTo(200, document.body.scrollHeight);")
        # check the average rating by using the values found here

        productTitleList = []
        reviewURLList = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []

        for review in reviews:

            productTitleList.append(product_df['Title'][0])
            reviewURLList.append(product_df['ReviewsURL'][0])

            # need to fix this, shouldn't need both try and if
            try:
                if (review.find('h3', class_='review-title').text):
                    title = review.find('h3', class_='review-title').text
                    reviewTitleList.append(title)
        #             print(title)
                elif (review.find('h3', class_='review-title').text) == 'None':
                    title = 'None'
                    reviewTitleList.append(title)
        #             print(title)
                else:
                    # if I can get this part to work ..
                    # this is where I can add the scroll down
                    # then try to grab the title again
                    pass

            except:
                pass

            # adding try only because the browser needs to scroll
            # once this is added, there wont need to be a try
            # each review has to have a star value
            try:
                reviewRating = review.find('span', class_='seo-avg-rating').text
                reviewRatingList.append(reviewRating)
        #         print(reviewRating)
            except:
                pass

            # adding try and except for the review body for same reason as stars
            try:
                reviewComment = review.find('p').text
                reviewCommentList.append(reviewComment)
        #         print(reviewComment)
            except:
                pass

        data = {
            'ProductTitle': productTitleList,
            'ReviewURL': reviewURLList,
            'ReviewTitle': reviewTitleList,
            'ReviewStarRating': reviewRatingList,
            'ReviewComment': reviewCommentList
        }

        productReview_df = pd.DataFrame(data)
    #     productReview_df.head()
    
        review_df = review_df.append(productReview_df)
    
    except:
        pass

In [10]:
# len(review_df)
review_df.head()

,ProductTitle,ReviewURL,ReviewTitle,ReviewStarRating,ReviewComment
0,"Filtrete by 3M Room Air Purifier, Console, 110...",/ip/Filtrete-by-3M-Room-Air-Purifier-Console-1...,Wonderful!,5.0,"Love it, has reduced allergens"
1,"Filtrete by 3M Room Air Purifier, Console, 110...",/ip/Filtrete-by-3M-Room-Air-Purifier-Console-1...,Filter,5.0,Easy to assemble. Still haven't had to change ...
2,"Filtrete by 3M Room Air Purifier, Console, 110...",/ip/Filtrete-by-3M-Room-Air-Purifier-Console-1...,Works Its Heart Out,4.0,I like everything about this unit EXCEPT the p...
0,"Filtrete by 3M Room Air Purifier, Console, 110...",/ip/Filtrete-by-3M-Room-Air-Purifier-Console-1...,Happy Holmes customer,5.0,Very happy with the unit. I wanted to give it...
1,"Filtrete by 3M Room Air Purifier, Console, 110...",/ip/Filtrete-by-3M-Room-Air-Purifier-Console-1...,Desktop Air Purifiers and Filters,5.0,Air purifier works great. Comes with one filte...


In [ ]:
reviews = soup.find('div', class_='review-highlight')
positive = reviews.find('div', class_='font-bold highlight-title').text
stars = reviews.find('span', class_='seo-avg-rating').text
# starts = reviews.find('span', class_='seo-average-rating')
body = reviews.find('div', class_='collapsable-content-container').text
print(positive)
print(stars)
print(body)

In [ ]:
reviews = soup.find_all('div', class_='ReviewList-content')

In [ ]:
# need to add a way to make sure I am capturing all of the reviews
# can check this by looking at NumberofReview for this ReviewsRUL
# to scroll down a little use the below command
# browser.execute_script("window.scrollTo(200, document.body.scrollHeight);")
# check the average rating by using the values found here

productTitleList = []
reviewURLList = []
reviewTitleList = []
reviewRatingList = []
reviewCommentList = []

for review in reviews:
    
    productTitleList.append(product_df['Title'][0])
    reviewURLList.append(product_df['ReviewsURL'][0])
    
    # need to fix this, shouldn't need both try and if
    try:
        if (review.find('h3', class_='review-title').text):
            title = review.find('h3', class_='review-title').text
            reviewTitleList.append(title)
#             print(title)
        elif (review.find('h3', class_='review-title').text) == 'None':
            title = 'None'
            reviewTitleList.append(title)
#             print(title)
        else:
            # if I can get this part to work ..
            # this is where I can add the scroll down
            # then try to grab the title again
            pass
        
    except:
        pass
    
    # adding try only because the browser needs to scroll
    # once this is added, there wont need to be a try
    # each review has to have a star value
    try:
        reviewRating = review.find('span', class_='seo-avg-rating').text
        reviewRatingList.append(reviewRating)
#         print(reviewRating)
    except:
        pass
    
    # adding try and except for the review body for same reason as stars
    try:
        reviewComment = review.find('p').text
        reviewCommentList.append(reviewComment)
#         print(reviewComment)
    except:
        pass
    
ddata = {
    'ProductTitle': productTitleList,
    'ReviewURL': reviewURLList,
    'ReviewTitle': reviewTitleList,
    'ReviewStarRating': reviewRatingList,
    'ReviewComment': reviewCommentList
}

review_df = pd.DataFrame(data)
review_df.head()

In [ ]:
browser.quit()

In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")